<a href="https://colab.research.google.com/github/Carlosmario90/ISYS5002_Assesments/blob/main/Assessment_Weather_App_Final_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📦 Setup and Configuration

Import required packages and setup environment.

In [4]:
!pip install pyinputplus
import matplotlib.pyplot as plt
import pyinputplus as pyip
import pytz
!pip install ipywidgets pandas matplotlib

!pip install fetch-my-weather
import fetch_my_weather
from fetch_my_weather import get_weather

!pip install hands-on-ai
from hands_on_ai.chat import get_response
import requests

!pip install dateparser
import dateparser
import re
from datetime import datetime, timedelta

# Set your timezone — change 'Australia/Perth' or 'Australia/Sydney' as needed
local_tz = pytz.timezone('Australia/Perth')
today = datetime.now(local_tz).date()

!pip install ipywidgets
from ipywidgets import widgets, VBox, HBox, Output, Button, ToggleButtons, Layout, Label, HTML
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt



## **Section 1 - 🌤️ Weather Data Functions**

This section handles the retrieval and processing of weather data for a given location.

The function extracts:

📍 Current weather conditions (temperature, humidity, windspeed)

📆 Multi-day forecast including temperature ranges, precipitation, sunrise/sunset, and other

In [5]:
def get_weather_data(location, forecast_days):
    data = fetch_my_weather.get_weather(location=location, format="raw_json")

    current = data.get("current_condition", [{}])[0]
    raw_forecast = data.get("weather", [])

    if len(raw_forecast) < forecast_days:
        print(f"⚠️ Only {len(raw_forecast)} days of forecast data are available (not {forecast_days}).")

    forecast = raw_forecast[:forecast_days]

    return {
        "location": location,
        "current_conditions": {
            "temperature_C": current.get("temp_C"),
            "humidity": current.get("humidity"),
            "windspeed_kmph": current.get("windspeedKmph"),
            "weather": current["weatherDesc"][0]["value"]
        },
        "forecast": [
            {
                "date": day.get("date"),
                "max_temp_C": day.get("maxtempC"),
                "min_temp_C": day.get("mintempC"),
                "avg_temp_C": day.get("avgtempC"),
                "sunrise": day.get("astronomy", [{}])[0].get("sunrise"),
                "sunset": day.get("astronomy", [{}])[0].get("sunset"),
                "condition": day.get("hourly", [{}])[0].get("weatherDesc", [{}])[0].get("value"),
                "humidity": day.get("hourly", [{}])[0].get("humidity"),
                "windspeed_kmph": day.get("hourly", [{}])[0].get("windspeedKmph"),
                "precipMM": sum(float(h.get("precipMM", "0") or "0") for h in day.get("hourly", []))
            }
            for day in forecast
        ]
    }